In [35]:
"""
DATE: 8/29/2020
AUTHOR: DANIEL WU
PURPOSE: Scrape Carmax Store Locations
LAST UPDATE:
"""

import requests
from bs4 import BeautifulSoup

from requests_html import HTMLSession
from requests_html import HTML

import os
import time
import datetime
import pandas as pd
import json
import re

def pause():
    programPause = input("Press the <ENTER> key to continue or type any letter to break loop")
    return programPause

In [130]:
root_dir = '/Users/user/Desktop/Jupyter Notebook'

root_url = 'https://carmax.com'
sub1_url = '/stores'

headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36',
           "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": 
           "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "Accept-Language": "en-US,en;q=0.5","Accept-Encoding": 
           "gzip, deflate"}

In [131]:
# Website Information at the State-Level

def req(subpath):
    r = requests.get(root_url + subpath, headers=headers)
    html = r.content
    soup = BeautifulSoup(html)
    return soup

soup = req(sub1_url)

st_match = soup.find_all('a', class_="stores-list-cabinet--single")

In [275]:
# City-level

addr = []

def srch(soup_obj):    
    for obj in soup_obj:        
        sub_url = obj['href']
        city_soup = req(sub_url)
        addr_match = city_soup.find('a', class_="kmx-typography--body-2 sdp-clickable-address")
        try:
            addr.append(str(addr_match.text))         
        except:
            pass 

for state in st_match:        
    
    sub2_url = state['href']    
    st_soup = req(sub2_url)    
    city_match1 = st_soup.find_all('a', class_="stores-list-cabinet--single")        
    
    try:
        city_match2 = st_soup.find_all('a', class_="store-link kmx-typography--display-2 kmx-button")         
        srch(city_match2)        
    except:
        pass 
        
    srch(city_match1)
    
addr_clean = [x.replace('\r\n', '') for x in addr]
addr_clean = [" ".join(re.split("\s+", x, flags=re.UNICODE)) for x in addr_clean]
addr_clean = [x.strip() for x in addr_clean]

TypeError: 'str' object cannot be interpreted as an integer

In [295]:
street_addr = []
state_zip = []

addr_split = [x.split(',') for x in addr_clean]

for i in addr_split:
    street_addr.append(i[0])
    state_zip.append(i[1].strip())

zip_split = [x.split(' ') for x in state_zip]

state = []
zipcode = []

for i in zip_split:    
    state.append(i[0])
    zipcode.append(i[1])
    


In [299]:
df = pd.DataFrame()

df['Street_Addr'] = street_addr
df['State'] = state
df['Zip'] = zipcode

In [302]:
df.to_excel("carmax_addr.xlsx")